In [1]:
import pandas as pd
import numpy as np
from glob import glob #read directory system to import files
import re
import nltk

%matplotlib inline

import os
os.chdir('C:\\Users\\dwagn\\Downloads\\Project\\Project')

In [2]:
OHCO = ['book_id','chap_num', 'para_num', 'sent_num', 'token_num']
epub_dir = 'epubs'

In [3]:
roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
letters = '[A-Za-z*]+'
chap_pats = {
    121: {
        'start_line': 23,
        'end_line': 8019,
        'book': re.compile('^\s*BOOK\s+{}\.$'.format(roman)),
        'chapter': re.compile("^CHAPTER\s+\d+$")
    },
     42671: {
        'start_line': 25,
        'end_line': 13356,
        'book': re.compile('^\s*BOOK\s+{}\.$'.format(roman)),
        'chapter': re.compile('^\s*CHAPTER\s+{}\.$'.format(roman))
    },
     141: {
        'start_line': 23,
        'end_line': 15702,
        'book': re.compile('^\s*BOOK\s+{}\.$'.format(roman)),
        'chapter': re.compile('^\s*CHAPTER\s+{}$'.format(roman))
     }
}

In [4]:
def acquire_epubs(epub_list, chap_pats, OHCO=OHCO):
    
    my_lib = []
    my_doc = []

    for epub_file in epub_list:
        
        # Get PG ID from filename
        book_id = int(epub_file.split('-')[-1].split('.')[0].replace('pg',''))
        print("BOOK ID", book_id)
        
        # Import file as lines
        lines = open(epub_file, 'r', encoding='utf-8-sig').readlines()
        df = pd.DataFrame(lines, columns=['line_str'])
        df.index.name = 'line_num'
        df.line_str = df.line_str.str.strip()
        df['book_id'] = book_id
        
        # FIX CHARACTERS TO IMPROVE TOKENIZATION
        df.line_str = df.line_str.str.replace('—', ' — ')
        
        # Get book title and put into LIB table -- note problems, though
        book_title = re.sub(r"The Project Gutenberg eBook( of|,) ", "", df.loc[0].line_str, flags=re.IGNORECASE)
        book_title = re.sub(r"Project Gutenberg's ", "", book_title, flags=re.IGNORECASE)
        
        # Remove cruft
        a = chap_pats[book_id]['start_line'] - 1
        b = chap_pats[book_id]['end_line'] + 1
        df = df.iloc[a:b]
        
        # Chunk by chapter
        chap_lines = df.line_str.str.match(chap_pats[book_id]['chapter'])
        chap_nums = [i+1 for i in range(df.loc[chap_lines].shape[0])]
        df.loc[chap_lines, 'chap_num'] = chap_nums
        df.chap_num = df.chap_num.ffill()

        # Clean up
        df = df[~df.chap_num.isna()] # Remove chapter heading lines
        df = df.loc[~chap_lines] # Remove everything before Chapter 1
        df['chap_num'] = df['chap_num'].astype('int')
        
        # Group -- Note that we exclude the book level in the OHCO at this point
        df = df.groupby(OHCO[1:2]).line_str.apply(lambda x: '\n'.join(x)).to_frame() # Make big string
        
        # Split into paragrpahs
        df = df['line_str'].str.split(r'\n\n+', expand=True).stack().to_frame().rename(columns={0:'para_str'})
        df.index.names = OHCO[1:3] # MAY NOT BE NECESSARY UNTIL THE END
        df['para_str'] = df['para_str'].str.replace(r'\n', ' ').str.strip()
        df = df[~df['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
        
        # Set index
        df['book_id'] = book_id
        df = df.reset_index().set_index(OHCO[:3])

        # Register
        my_lib.append((book_id, book_title, epub_file))
        my_doc.append(df)

    docs = pd.concat(my_doc)
    library = pd.DataFrame(my_lib, columns=['book_id', 'book_title', 'book_file']).set_index('book_id')
    return library, docs

In [5]:
epubs = [epub for epub in sorted(glob(epub_dir+'/*.txt'))]
LIB, DOC = acquire_epubs(epubs, chap_pats)

BOOK ID 141
BOOK ID 121
BOOK ID 42671


In [20]:
DOC

para_str
book_id chap_num para_num                                                   
141     49       1         About thirty years ago Miss Maria Ward, of Hun...
                 2         Their homes were so distant, and the circles i...
                 3         The letter was not unproductive. It re-establi...
                 4         Such were its immediate effects, and within a ...
                 5         Sir Thomas could not give so instantaneous and...
...                                                                      ...
42671   61       15        With the Gardiners, they were always on the mo...
                 16                        *       *       *       *       *
                 17                                      Transcriber's note:
                 18        Spelling and hyphen changes have been made so ...
                 19        ***END OF THE PROJECT GUTENBERG EBOOK PRIDE AN...

[4897 rows x 1 columns]

In [6]:
LIB

,book_title,book_file
book_id,,
141,"Mansfield Park, by Jane Austen",epubs/AUSTEN_JANE_MANSFIELD_PARK-pg141.txt
121,"Northanger Abbey, by Jane Austen",epubs/AUSTEN_JANE_NORTHANGER_ABBEY-pg121.txt
42671,"Pride and Prejudice, by Jane Austen, Edited",epubs/AUSTEN_JANE_PRIDE_AND_PREJUDICE-pg42671.txt


Tokenize

In [7]:
def tokenize(doc_df, remove_pos_tuple=False, OHCO=OHCO):
    
    # Paragraphs to Sentences
    df = doc_df.para_str\
        .apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'sent_str'})
    
    # Sentences to Tokens
    # .apply(lambda x: pd.Series(nltk.pos_tag(nltk.word_tokenize(x))))\
    df = df.sent_str\
        .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
    
    # Add index
    df.index.names = OHCO
    
    return df

In [8]:
TOKEN = tokenize(DOC)

In [9]:
TOKEN.sample(25)

pos_tuple   pos  \
book_id chap_num para_num sent_num token_num                               
42671   54       29       0        10                     (to, TO)    TO   
        44       10       0        43                     (or, CC)    CC   
        54       27       0        20                     (be, VB)    VB   
        18       70       0        22                  (her, PRP$)  PRP$   
121     54       2        7        37              (different, JJ)    JJ   
141     95       30       10       4              (astonished, JJ)    JJ   
        82       25       1        1                 (myself,, NN)    NN   
42671   42       3        3        2            (consequently, RB)    RB   
141     66       4        1        45                    (all, DT)    DT   
        91       4        4        27                    (the, DT)    DT   
42671   36       6        2        25                  (his, PRP$)  PRP$   
121     38       7        2        23                    (any, DT)    DT   
                 1        1        52                (however, RB)    RB   
141     53       39       2        7                     (the, DT)    DT   
121     51       1        0        30                    (own, JJ)    JJ   
        33       5        1        24                  (those, DT)    DT   
42671   6        48       1        1                    (mind, NN)    NN   
        43       58       2        7                   ("when, IN)    IN   
121     49       19       4        15                  (judge, VB)    VB   
141     62       16       0        6                  (wife,”, NN)    NN   
        83       21       8        8                     ((as, JJ)    JJ   
121     51       5        7        6          (circumstanced, VBN)   VBN   
42671   19       8        3        10                   (you, PRP)   PRP   
        43       52       3        9                    (last, JJ)    JJ   
121     55       10       1        5                      (be, VB)    VB   

                                                  token_str  
book_id chap_num para_num sent_num token_num                 
42671   54       29       0        10                    to  
        44       10       0        43                    or  
        54       27       0        20                    be  
        18       70       0        22                   her  
121     54       2        7        37             different  
141     95       30       10       4             astonished  
        82       25       1        1                myself,  
42671   42       3        3        2           consequently  
141     66       4        1        45                   all  
        91       4        4        27                   the  
42671   36       6        2        25                   his  
121     38       7        2        23                   any  
                 1        1        52               however  
141     53       39       2        7                    the  
121     51       1        0        30                   own  
        33       5        1        24                 those  
42671   6        48       1        1                   mind  
        43       58       2        7                  "when  
121     49       19       4        15                 judge  
141     62       16       0        6                 wife,”  
        83       21       8        8                    (as  
121     51       5        7        6          circumstanced  
42671   19       8        3        10                   you  
        43       52       3        9                   last  
121     55       10       1        5                     be

In [10]:
TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')

VOCAB = TOKEN.term_str.value_counts().to_frame().rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

VOCAB['num'] = VOCAB.term_str.str.match("\d+").astype('int')

In [11]:
VOCAB

,term_str,n,num
term_id,,,
0,,922,0
1,1,1,1
2,10,3,1
3,14th,2,1
4,15th,1,1
...,...,...,...
11878,youths,1,0
11879,youwill,1,0
11880,zeal,5,0


In [12]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [13]:
sw.sample(10)

,dummy
term_str,
then,1
mightn't,1
be,1
haven't,1
if,1
isn't,1
that,1
she,1
this,1


In [14]:
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [15]:
VOCAB[VOCAB.stop == 1].sample(10)

,term_str,n,num,stop
term_id,,,,
9759,some,639,0,1
4791,had,3521,0,1
4928,having,341,0,1
11616,while,229,0,1
11211,until,1,0,1
4450,further,29,0,1
10490,than,880,0,1
11516,we,749,0,1
5035,herself,662,0,1


Porter Stems

In [16]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
VOCAB['p_stem'] = VOCAB.term_str.apply(stemmer.stem)

VOCAB.sample(10)

,term_str,n,num,stop,p_stem
term_id,,,,,
11447,wan,1,0,0,wan
4121,finely,2,0,0,fine
6198,librarytheir,1,0,0,librarytheir
3631,envyings,1,0,0,envi
11181,unsettled,8,0,0,unsettl
10642,tis,10,0,0,ti
179,acutest,1,0,0,acutest
10613,tidings,9,0,0,tide
11505,watsons,1,0,0,watson


In [17]:
TOKEN

pos_tuple  pos  \
book_id chap_num para_num sent_num token_num                             
141     49       1        0        0                  (About, IN)   IN   
                                   1                 (thirty, CD)   CD   
                                   2                 (years, NNS)  NNS   
                                   3                    (ago, RB)   RB   
                                   4                  (Miss, NNP)  NNP   
...                                                           ...  ...   
42671   61       19       0        4             (GUTENBERG, NNP)  NNP   
                                   5                 (EBOOK, NNP)  NNP   
                                   6                 (PRIDE, NNP)  NNP   
                                   7                   (AND, NNP)  NNP   
                                   8          (PREJUDICE***, NNP)  NNP   

                                                 token_str   term_str  
book_id chap_num para_num sent_num token_num                           
141     49       1        0        0                 About      about  
                                   1                thirty     thirty  
                                   2                 years      years  
                                   3                   ago        ago  
                                   4                  Miss       miss  
...                                                    ...        ...  
42671   61       19       0        4             GUTENBERG  gutenberg  
                                   5                 EBOOK      ebook  
                                   6                 PRIDE      pride  
                                   7                   AND        and  
                                   8          PREJUDICE***  prejudice  

[359903 rows x 4 columns]

In [18]:
token1 = TOKEN
pos_max = token1.groupby(['term_str',"pos"]).count().sort_values("token_str", ascending = False).groupby(level=0).head(1)\
    .reset_index().set_index('term_str')
pos_max.sort_index().tail(200)
VOCAB['pos_max'] = VOCAB.term_str.map(pos_max.pos)
VOCAB

,term_str,n,num,stop,p_stem,pos_max
term_id,,,,,,
0,,922,0,0,,NN
1,1,1,1,0,1,JJ
2,10,3,1,0,10,NN
3,14th,2,1,0,14th,CD
4,15th,1,1,0,15th,CD
...,...,...,...,...,...,...
11878,youths,1,0,0,youth,NNS
11879,youwill,1,0,0,youwil,VB
11880,zeal,5,0,0,zeal,NN


save csvs

In [19]:
DOC.to_csv('DOC.csv')
LIB.to_csv('LIB.csv')
VOCAB.to_csv('VOCAB.csv')
TOKEN.to_csv('TOKEN.csv')